This notebook trains and tests an SCA agent on the Laser Hockey Enviroment

In [1]:
# Imports
import numpy as np
import laserhockey.hockey_env as h_env
from importlib import reload
from SAC import Agent
import matplotlib.pyplot as plt
# from gym import wrappers

In [2]:
# Global variables
n_games = 100
load_checkpoint = True

In [3]:
# Initialization 
np.set_printoptions(suppress=True)
reload(h_env)

# get enviroment
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_DEFENSE)

#init agent
agent = Agent(input_dims=env.observation_space.shape, 
              env=env,
              n_actions=int(env.action_space.shape[0]/2))

# and oponent
opponent = h_env.BasicOpponent()


/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [4]:
# Plotting function
def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)

In [5]:
# Training of agent

# uncomment this line and do a mkdir tmp && mkdir video if you want to
# record video of the agent playing the game.
#env = wrappers.Monitor(env, 'tmp/video', video_callable=lambda episode_id: True, force=True)

best_score = env.reward_range[0]
score_history = []

if not load_checkpoint:
    for i in range(n_games):
        observation, info = env.reset()
        obs_opponent = env.obs_agent_two()
        
        done = False
        score = 0
        
        while not done:
            action1 = agent.act(observation)
            action2 = opponent.act(obs_opponent)
            
            observation_, reward, done, _, info = env.step(np.hstack([action1,action2]))
            score += reward
            agent.remember(observation, action1, reward, observation_, done)
            agent.learn()
                
            obs_opponent = env.obs_agent_two()
            observation = observation_
            
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score:
            best_score = avg_score
            if not load_checkpoint:
                agent.save_models()

        print('episode ', i, 'score %.1f' % score, 'avg_score %.1f' % avg_score)

    x = [i+1 for i in range(n_games)]
    plot_learning_curve(x, score_history, 'SCA.png')

else: 
    agent.load_models()
    player1 = h_env.HumanOpponent(env=env, player=1)
    player2 = agent
    env.render()   
    obs, info = env.reset()
    obs_agent2 = env.obs_agent_two()
    for _ in range(5000):
        env.render()
        a1 = player1.act(obs) 
        a2 = player2.act(obs_agent2)
        obs, r, d, _, info = env.step(np.hstack([a1,a2]))    
        obs_agent2 = env.obs_agent_two()
        if d: break


.... loading models ....
Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s
 shoot :	space


/Users/emilbreustedt/Documents/GitHub/RL-Hockey/SAC.py:201: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  state = T.Tensor([observation]).to(self.actor.device)


: 